# Data Preparation 3

This notebook consist in the third data preparation.

The goal of this notebook is to fill the main Dataframe with Invasive and Non-invasive Ventilation features from PROCEDUREEVENTS_MV.csv file.

In [393]:
import pandas as pd
from IPython.display import display
import numpy as np
import sqlite3
pd.options.display.max_columns = None
pd.options.display.max_rows = 200

In [394]:
admissions=pd.read_csv('MAIN_DF_2.csv')
admissions=admissions.drop(['Unnamed: 0'], axis=1)
admissions=admissions.reset_index(drop=True)
print(admissions.shape)
print('unique SUBJECT_ID:', admissions.SUBJECT_ID.nunique())
print('unique HADM_ID   :', admissions.HADM_ID.nunique())
print('unique ICUSTAY_ID:', admissions.ICUSTAY_ID.nunique())
admissions.head()

(48989, 28)
unique SUBJECT_ID: 36659
unique HADM_ID   : 46273
unique ICUSTAY_ID: 48989


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,GENDER,AGE_AD,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,ETHNICITY,DIAGNOSIS,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS,LOS_C,TIMEDELTA,TIMEDELTA_C,HOSPITAL_EXPIRE_FLAG,HEIGHT,WEIGHT,BMI
0,3,145834,211552,M,76.53,2101-10-20 19:08:00,2101-10-31 13:58:00,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,WHITE,HYPOTENSION,carevue,MICU,MICU,12,12,2101-10-20 19:10:11,2101-10-26 20:43:09,6.0646,>4,10.78,>10,0,NaN,NaN,NaN
1,4,185777,294638,F,47.84,2191-03-16 00:28:00,2191-03-23 18:41:00,EMERGENCY,EMERGENCY ROOM ADMIT,HOME WITH HOME IV PROVIDR,Private,WHITE,"FEVER,DEHYDRATION,FAILURE TO THRIVE",carevue,MICU,MICU,52,52,2191-03-16 00:29:31,2191-03-17 16:46:31,1.6785,1-2,7.76,6-10,0,NaN,NaN,NaN
2,6,107064,228232,F,65.94,2175-05-30 07:15:00,2175-06-15 16:00:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,WHITE,CHRONIC RENAL FAILURE/SDA,carevue,SICU,SICU,33,33,2175-05-30 21:30:54,2175-06-03 13:39:54,3.6729,2-4,16.36,>10,0,NaN,NaN,NaN
3,9,150750,220597,M,41.79,2149-11-09 13:06:00,2149-11-14 10:15:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicaid,NW,HEMORRHAGIC CVA,carevue,MICU,MICU,15,15,2149-11-09 13:07:02,2149-11-14 20:52:14,5.3231,>4,4.88,3-6,1,NaN,NaN,NaN
4,11,194540,229441,F,50.15,2178-04-16 06:18:00,2178-05-11 19:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Private,WHITE,BRAIN MASS,carevue,SICU,SICU,57,57,2178-04-16 06:19:32,2178-04-17 20:21:05,1.5844,1-2,25.53,>10,0,NaN,NaN,NaN


Physicians from Hospital selected from D_ITEMS.csv file some variables of interest. They defined a column called EXTRACT where it inform us if the variable is for our interest or not.

This is the file with the information of the variables of interest.

The goal is to extract them all and aggregate in the main dataframe in a proper way. First we are going to focus in the variables corresponding to the admission, the ones that were registered first.

In [395]:
items=pd.read_excel('ITEMS_ID.xlsx')
items=items.drop(['ROW_ID'], axis=1)
items=items[items['EXTRACT']==1]
print(items.shape)
items=items.reset_index(drop=True)
items

(186, 9)


,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,EXTRACT
0,225792,Invasive Ventilation,Invasive Ventilation,metavision,procedureevents_mv,2-Ventilation,None,Process,1.0
1,225794,Non-invasive Ventilation,Non-invasive Ventilation,metavision,procedureevents_mv,2-Ventilation,None,Process,1.0
2,227456,Albumin,Albumin,metavision,chartevents,Labs,None,Numeric with tag,1.0
3,225624,BUN,BUN,metavision,chartevents,Labs,None,Numeric,1.0
4,220615,Creatinine,Creatinine,metavision,chartevents,Labs,None,Numeric,1.0
5,225651,Direct Bilirubin,Direct Bilirubin,metavision,chartevents,Labs,None,Numeric,1.0
6,220621,Glucose (serum),Glucose (serum),metavision,chartevents,Labs,None,Numeric,1.0
7,226537,Glucose (whole blood),Glucose (whole blood),metavision,chartevents,Labs,None,Numeric,1.0
8,228388,Glucose (whole blood) (soft),Glucose (whole blood) (soft),metavision,chartevents,Labs,None,Numeric with tag,1.0
9,225664,Glucose finger stick,Glucose finger stick,metavision,chartevents,Labs,None,Numeric,1.0


In [396]:
items.LINKSTO.value_counts()

chartevents           184
procedureevents_mv      2
Name: LINKSTO, dtype: int64

The source of this variables mainly are located in the chartevents database

In [397]:
items.ITEMID.nunique()

186

We do the aggregation via ITEMID because the unicity

First we are going to deal with the ventilation items

## Invasive Ventilation extraction

In [398]:
df=pd.read_csv('data/PROCEDUREEVENTS_MV.csv')
df=df[df['ITEMID']==items.ITEMID[0]]
print(df.shape)
print('unique ICUSTAY_ID:', df.ICUSTAY_ID.nunique())
df.head()

(10749, 25)
unique ICUSTAY_ID: 9118


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,ITEMID,VALUE,VALUEUOM,LOCATION,LOCATIONCATEGORY,STORETIME,CGID,ORDERID,LINKORDERID,ORDERCATEGORYNAME,SECONDARYORDERCATEGORYNAME,ORDERCATEGORYDESCRIPTION,ISOPENBAG,CONTINUEINNEXTDEPT,CANCELREASON,STATUSDESCRIPTION,COMMENTS_EDITEDBY,COMMENTS_CANCELEDBY,COMMENTS_DATE
2,381,29070,115071,232563.0,2145-03-12 23:05:00,2145-03-18 20:01:00,225792,8456.0,hour,NaN,NaN,2145-03-18 20:01:39,20889,2383835,2383835,Ventilation,NaN,Task,1,0,0,FinishedRunning,NaN,NaN,NaN
23,402,31608,152365,237055.0,2133-01-10 01:00:00,2133-02-03 12:37:00,225792,35257.0,hour,NaN,NaN,2133-02-03 12:37:03,20889,1377686,1377686,Ventilation,NaN,Task,1,0,0,FinishedRunning,NaN,NaN,NaN
89,4,30354,120396,243526.0,2154-12-24 12:06:00,2154-12-26 17:00:00,225792,3174.0,min,NaN,NaN,2154-12-26 17:00:45,20889,9613331,9613331,Ventilation,NaN,Task,1,0,0,FinishedRunning,NaN,NaN,NaN
142,57,18082,181163,267692.0,2156-02-24 11:38:00,2156-02-27 14:00:00,225792,4462.0,min,NaN,NaN,2156-02-27 15:03:00,20290,5651463,5651463,Ventilation,NaN,Task,1,0,0,Stopped,NaN,NaN,NaN
188,103,27143,189921,258818.0,2189-12-09 13:00:00,2189-12-14 13:34:00,225792,7234.0,hour,NaN,NaN,2189-12-14 13:35:00,15225,3374231,3374231,Ventilation,NaN,Task,1,0,0,FinishedRunning,NaN,NaN,NaN


In [399]:
df.isnull().sum()

ROW_ID                            0
SUBJECT_ID                        0
HADM_ID                           0
ICUSTAY_ID                        4
STARTTIME                         0
ENDTIME                           0
ITEMID                            0
VALUE                             0
VALUEUOM                          0
LOCATION                      10749
LOCATIONCATEGORY              10749
STORETIME                         0
CGID                              0
ORDERID                           0
LINKORDERID                       0
ORDERCATEGORYNAME                 0
SECONDARYORDERCATEGORYNAME    10749
ORDERCATEGORYDESCRIPTION          0
ISOPENBAG                         0
CONTINUEINNEXTDEPT                0
CANCELREASON                      0
STATUSDESCRIPTION                 0
COMMENTS_EDITEDBY             10699
COMMENTS_CANCELEDBY           10492
COMMENTS_DATE                 10442
dtype: int64

In [400]:
df['STATUSDESCRIPTION'].value_counts()

FinishedRunning    10184
Rewritten            380
Stopped              169
Paused                16
Name: STATUSDESCRIPTION, dtype: int64

In [401]:
df=df.drop(['ROW_ID', 'ITEMID', 'LOCATION', 'LOCATIONCATEGORY', 'STORETIME', 'CGID',
            'ORDERID', 'LINKORDERID', 'ORDERCATEGORYNAME', 
           'SECONDARYORDERCATEGORYNAME', 'ORDERCATEGORYDESCRIPTION', 'ISOPENBAG', 
            'CONTINUEINNEXTDEPT', 'CANCELREASON', 'STATUSDESCRIPTION', 
            'COMMENTS_EDITEDBY', 'COMMENTS_CANCELEDBY', 'COMMENTS_DATE'], axis=1)

In [402]:
print(df.shape)
print('unique ICUSTAY_ID:', df.ICUSTAY_ID.nunique())
df.head()

(10749, 7)
unique ICUSTAY_ID: 9118


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,VALUE,VALUEUOM
2,29070,115071,232563.0,2145-03-12 23:05:00,2145-03-18 20:01:00,8456.0,hour
23,31608,152365,237055.0,2133-01-10 01:00:00,2133-02-03 12:37:00,35257.0,hour
89,30354,120396,243526.0,2154-12-24 12:06:00,2154-12-26 17:00:00,3174.0,min
142,18082,181163,267692.0,2156-02-24 11:38:00,2156-02-27 14:00:00,4462.0,min
188,27143,189921,258818.0,2189-12-09 13:00:00,2189-12-14 13:34:00,7234.0,hour


In [403]:
df.isnull().sum()

SUBJECT_ID    0
HADM_ID       0
ICUSTAY_ID    4
STARTTIME     0
ENDTIME       0
VALUE         0
VALUEUOM      0
dtype: int64

In [404]:
df=df.dropna()
print(df.shape)

(10745, 7)


In [405]:
a=pd.to_datetime(df.ENDTIME)-pd.to_datetime(df.STARTTIME)
np.round(a.dt.total_seconds()*24*60/86400, 2).head()

2       8456.0
23     35257.0
89      3174.0
142     4462.0
188     7234.0
dtype: float64

VALUEUOM units are incorrect...

In [406]:
df['VALUE']=np.round(a.dt.total_seconds()*24*60/86400, 0)
df=df.rename({'VALUE':'MINUTES_IV'}, axis='columns')
df=df.drop(['VALUEUOM'], axis=1)

In [407]:
print(df.shape)
print('unique ICUSTAY_ID:', df.ICUSTAY_ID.nunique())
df.head()

(10745, 6)
unique ICUSTAY_ID: 9118


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,MINUTES_IV
2,29070,115071,232563.0,2145-03-12 23:05:00,2145-03-18 20:01:00,8456.0
23,31608,152365,237055.0,2133-01-10 01:00:00,2133-02-03 12:37:00,35257.0
89,30354,120396,243526.0,2154-12-24 12:06:00,2154-12-26 17:00:00,3174.0
142,18082,181163,267692.0,2156-02-24 11:38:00,2156-02-27 14:00:00,4462.0
188,27143,189921,258818.0,2189-12-09 13:00:00,2189-12-14 13:34:00,7234.0


In [408]:
df['ICUSTAY_ID'].value_counts().head()

249772.0    14
270267.0    11
270546.0    11
262458.0    10
275466.0     9
Name: ICUSTAY_ID, dtype: int64

We have ICUSTAY_ID with several invasive ventilation, we are going to take the maximum value ww have repetitions due to errors in the data registration...

In [409]:
df[df['ICUSTAY_ID']==249772.0]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,MINUTES_IV
5569,27469,126534,249772.0,2174-09-07 02:00:00,2174-09-07 09:31:00,451.0
5570,27469,126534,249772.0,2174-09-07 02:00:00,2174-09-07 09:00:00,420.0
5674,27469,126534,249772.0,2174-09-07 14:00:00,2174-09-08 09:30:00,1170.0
5677,27469,126534,249772.0,2174-09-08 12:08:00,2174-09-08 14:50:00,162.0
5679,27469,126534,249772.0,2174-09-08 15:33:00,2174-09-09 08:28:00,1015.0
5680,27469,126534,249772.0,2174-09-08 15:33:00,2174-09-22 01:28:00,19315.0
5681,27469,126534,249772.0,2174-09-09 08:28:00,2174-09-10 03:58:00,1170.0
5682,27469,126534,249772.0,2174-09-09 08:50:00,2174-09-10 04:20:00,1170.0
5684,27469,126534,249772.0,2174-09-10 03:45:00,2174-09-10 23:15:00,1170.0
5685,27469,126534,249772.0,2174-09-10 03:54:00,2174-09-10 23:24:00,1170.0


In [410]:
#df['MINUTES_IV'] = df.groupby(['SUBJECT_ID', 'HADM_ID','ICUSTAY_ID'])['MINUTES_IV'].transform('sum')
df=df.sort_values(by=['MINUTES_IV'], ascending=False)
df=df.drop_duplicates(['SUBJECT_ID', 'HADM_ID','ICUSTAY_ID'])
df.shape

(9118, 6)

In [411]:
df[df['ICUSTAY_ID']==249772.0]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,MINUTES_IV
5680,27469,126534,249772.0,2174-09-08 15:33:00,2174-09-22 01:28:00,19315.0


In [412]:
df=df.drop(['STARTTIME', 'ENDTIME'], axis=1)
print(df.shape)
print('unique ICUSTAY_ID:', df.ICUSTAY_ID.nunique())
df.head()

(9118, 4)
unique ICUSTAY_ID: 9118


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,MINUTES_IV
60938,78076,138363,257336.0,135267.0
27417,58662,163361,253584.0,112281.0
214215,49555,111955,234725.0,111747.0
172412,69745,145211,280112.0,94482.0
71026,77614,140792,251027.0,86898.0


In [413]:
admissions=pd.merge(admissions,df,how='left',on=['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID'])

## Non-invasive Ventilation extraction

In [414]:
df=pd.read_csv('data/PROCEDUREEVENTS_MV.csv')
df=df[df['ITEMID']==items.ITEMID[1]]
print(df.shape)
print('unique ICUSTAY_ID:', df.ICUSTAY_ID.nunique())
df.head()

(1411, 25)
unique ICUSTAY_ID: 1009


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,ITEMID,VALUE,VALUEUOM,LOCATION,LOCATIONCATEGORY,STORETIME,CGID,ORDERID,LINKORDERID,ORDERCATEGORYNAME,SECONDARYORDERCATEGORYNAME,ORDERCATEGORYDESCRIPTION,ISOPENBAG,CONTINUEINNEXTDEPT,CANCELREASON,STATUSDESCRIPTION,COMMENTS_EDITEDBY,COMMENTS_CANCELEDBY,COMMENTS_DATE
159,74,18082,181163,267692.0,2156-02-27 14:45:00,2156-02-29 06:58:00,225794,2413.0,hour,NaN,NaN,2156-02-29 12:48:00,19510,6729578,6729578,Ventilation,NaN,Task,1,0,2,Rewritten,RN,NaN,2156-02-29 12:59:00
160,75,18082,181163,267692.0,2156-02-27 14:45:00,2156-02-29 06:57:00,225794,2412.0,hour,NaN,NaN,2156-02-29 12:59:00,14695,9438042,9438042,Ventilation,NaN,Task,0,0,0,Rewritten,NaN,NaN,NaN
161,76,18082,181163,267692.0,2156-02-27 14:45:00,2156-02-29 07:00:00,225794,2415.0,hour,NaN,NaN,2156-02-29 13:00:00,19510,7219296,7219296,Ventilation,NaN,Task,1,0,0,FinishedRunning,NaN,NaN,NaN
162,77,18082,181163,267692.0,2156-02-29 08:30:00,2156-03-02 00:45:00,225794,2415.0,hour,NaN,NaN,2156-02-29 12:51:00,14695,2764224,2764224,Ventilation,NaN,Task,0,0,0,Rewritten,NaN,NaN,NaN
163,78,18082,181163,267692.0,2156-02-29 08:30:00,2156-02-29 08:32:00,225794,2.0,hour,NaN,NaN,2156-02-29 12:58:00,14695,2242821,6729578,Ventilation,NaN,Task,0,0,2,Rewritten,RN,NaN,2156-02-29 12:59:00


In [415]:
df=df.drop(['ROW_ID', 'ITEMID', 'LOCATION', 'LOCATIONCATEGORY', 'STORETIME', 'CGID',
            'ORDERID', 'LINKORDERID', 'ORDERCATEGORYNAME', 
           'SECONDARYORDERCATEGORYNAME', 'ORDERCATEGORYDESCRIPTION', 'ISOPENBAG', 
            'CONTINUEINNEXTDEPT', 'CANCELREASON', 'STATUSDESCRIPTION', 
            'COMMENTS_EDITEDBY', 'COMMENTS_CANCELEDBY', 'COMMENTS_DATE'], axis=1)

In [416]:
print(df.shape)
print('unique ICUSTAY_ID:', df.ICUSTAY_ID.nunique())
df.head()

(1411, 7)
unique ICUSTAY_ID: 1009


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,VALUE,VALUEUOM
159,18082,181163,267692.0,2156-02-27 14:45:00,2156-02-29 06:58:00,2413.0,hour
160,18082,181163,267692.0,2156-02-27 14:45:00,2156-02-29 06:57:00,2412.0,hour
161,18082,181163,267692.0,2156-02-27 14:45:00,2156-02-29 07:00:00,2415.0,hour
162,18082,181163,267692.0,2156-02-29 08:30:00,2156-03-02 00:45:00,2415.0,hour
163,18082,181163,267692.0,2156-02-29 08:30:00,2156-02-29 08:32:00,2.0,hour


In [417]:
df.isnull().sum()

SUBJECT_ID    0
HADM_ID       0
ICUSTAY_ID    0
STARTTIME     0
ENDTIME       0
VALUE         0
VALUEUOM      0
dtype: int64

In [418]:
a=pd.to_datetime(df.ENDTIME)-pd.to_datetime(df.STARTTIME)
np.round(a.dt.total_seconds()*24*60/86400, 0).head()

159    2413.0
160    2412.0
161    2415.0
162    2415.0
163       2.0
dtype: float64

VALUEUOM units are incorrect...

In [419]:
df['VALUE']=np.round(a.dt.total_seconds()*24*60/86400, 2)
df=df.rename({'VALUE':'MINUTES_NIV'}, axis='columns')
df=df.drop(['VALUEUOM'], axis=1)

In [420]:
print(df.shape)
print('unique ICUSTAY_ID:', df.ICUSTAY_ID.nunique())
df.head()

(1411, 6)
unique ICUSTAY_ID: 1009


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,MINUTES_NIV
159,18082,181163,267692.0,2156-02-27 14:45:00,2156-02-29 06:58:00,2413.0
160,18082,181163,267692.0,2156-02-27 14:45:00,2156-02-29 06:57:00,2412.0
161,18082,181163,267692.0,2156-02-27 14:45:00,2156-02-29 07:00:00,2415.0
162,18082,181163,267692.0,2156-02-29 08:30:00,2156-03-02 00:45:00,2415.0
163,18082,181163,267692.0,2156-02-29 08:30:00,2156-02-29 08:32:00,2.0


In [421]:
#df['MINUTES_NIV'] = df.groupby(['SUBJECT_ID', 'HADM_ID','ICUSTAY_ID'])['MINUTES_NIV'].transform('sum')
df=df.sort_values(by=['MINUTES_NIV'], ascending=False)
df=df.drop_duplicates(['SUBJECT_ID', 'HADM_ID','ICUSTAY_ID'])
df.shape

(1009, 6)

In [422]:
df=df.drop(['STARTTIME', 'ENDTIME'], axis=1)
print(df.shape)
print('unique ICUSTAY_ID:', df.ICUSTAY_ID.nunique())
df.head()

(1009, 4)
unique ICUSTAY_ID: 1009


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,MINUTES_NIV
7565,30202,117448,220559.0,20170.0
1065,31300,136581,203503.0,13116.0
119824,52622,157538,231245.0,10922.0
106263,63537,105106,288702.0,10772.0
46181,2378,162768,215412.0,10560.0


In [423]:
admissions=pd.merge(admissions,df,how='left',on=['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID'])

In [424]:
admissions['MINUTES_IV'] = admissions['MINUTES_IV'].transform(lambda x: x.fillna(0))
admissions['MINUTES_NIV'] = admissions['MINUTES_NIV'].transform(lambda x: x.fillna(0))

In [425]:
admissions['MINUTES_IV']=admissions['MINUTES_IV']/(admissions['LOS']*24*60)
admissions['MINUTES_NIV']=admissions['MINUTES_NIV']/(admissions['LOS']*24*60)

In [426]:
admissions['MINUTES_IV'] = np.where(admissions['MINUTES_IV']>1, 1, admissions['MINUTES_IV'])
admissions['MINUTES_NIV'] = np.where(admissions['MINUTES_NIV']>1, 1, admissions['MINUTES_NIV'])

In [427]:
admissions.head()

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,GENDER,AGE_AD,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,ETHNICITY,DIAGNOSIS,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS,LOS_C,TIMEDELTA,TIMEDELTA_C,HOSPITAL_EXPIRE_FLAG,HEIGHT,WEIGHT,BMI,MINUTES_IV,MINUTES_NIV
0,3,145834,211552,M,76.53,2101-10-20 19:08:00,2101-10-31 13:58:00,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,WHITE,HYPOTENSION,carevue,MICU,MICU,12,12,2101-10-20 19:10:11,2101-10-26 20:43:09,6.0646,>4,10.78,>10,0,NaN,NaN,NaN,0.0,0.0
1,4,185777,294638,F,47.84,2191-03-16 00:28:00,2191-03-23 18:41:00,EMERGENCY,EMERGENCY ROOM ADMIT,HOME WITH HOME IV PROVIDR,Private,WHITE,"FEVER,DEHYDRATION,FAILURE TO THRIVE",carevue,MICU,MICU,52,52,2191-03-16 00:29:31,2191-03-17 16:46:31,1.6785,1-2,7.76,6-10,0,NaN,NaN,NaN,0.0,0.0
2,6,107064,228232,F,65.94,2175-05-30 07:15:00,2175-06-15 16:00:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,WHITE,CHRONIC RENAL FAILURE/SDA,carevue,SICU,SICU,33,33,2175-05-30 21:30:54,2175-06-03 13:39:54,3.6729,2-4,16.36,>10,0,NaN,NaN,NaN,0.0,0.0
3,9,150750,220597,M,41.79,2149-11-09 13:06:00,2149-11-14 10:15:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicaid,NW,HEMORRHAGIC CVA,carevue,MICU,MICU,15,15,2149-11-09 13:07:02,2149-11-14 20:52:14,5.3231,>4,4.88,3-6,1,NaN,NaN,NaN,0.0,0.0
4,11,194540,229441,F,50.15,2178-04-16 06:18:00,2178-05-11 19:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Private,WHITE,BRAIN MASS,carevue,SICU,SICU,57,57,2178-04-16 06:19:32,2178-04-17 20:21:05,1.5844,1-2,25.53,>10,0,NaN,NaN,NaN,0.0,0.0


In [428]:
print(admissions.shape)
print('unique SUBJECT_ID:', admissions.SUBJECT_ID.nunique())
print('unique HADM_ID   :', admissions.HADM_ID.nunique())
print('unique ICUSTAY_ID:', admissions.ICUSTAY_ID.nunique())

(48989, 30)
unique SUBJECT_ID: 36659
unique HADM_ID   : 46273
unique ICUSTAY_ID: 48989


In [429]:
admissions.to_csv('MAIN_DF_3.csv')